In [1]:
import pandas as pd
import scanpy as sc
from scipy.spatial import cKDTree
from tqdm import tqdm
from pandas.api.types import CategoricalDtype
from scipy import sparse
import matplotlib.pyplot as plt
import anndata as ad
import numpy as np
import os
import time
from scipy.spatial import ConvexHull
from shapely.geometry import Point, Polygon
from shapely.prepared import prep

import warnings
warnings.filterwarnings('ignore')

In [3]:
raw_adata = sc.read_h5ad('/mnt/Data16Tc/home/haichao/code/SpaCon/Data/N_20231213_zxw/mouse_3/adata_processed.h5ad')

In [2]:
# path = '/mnt/Data18Td/Data/haichao/merfish_raw_data_zxw3/'
# decode_files_name = os.listdir(path + 'decode/')
# section_dict = {}
# for f in decode_files_name:
#     cellpose = pd.read_csv(path + 'cell_boundary/' + f[6:], index_col=0)
#     adata_tmp = raw_adata[raw_adata.obs.index.isin(list(set(cellpose.index) & set(raw_adata.obs.index)))]
#     print(f"{f}: {adata_tmp.obs['brain_section_label'].unique()}")
#     section_dict[f] = adata_tmp.obs['brain_section_label'].unique()

section_dict = {
    'spots_220713_wb3_sa1_B_13_20_5z18R_merfish5.csv': ['Zhuang-ABCA-3.013', 'Zhuang-ABCA-3.020']
, 'spots_220706_wb3_sa1_B_12_21_5z18R_merfish5.csv': ['Zhuang-ABCA-3.012', 'Zhuang-ABCA-3.021']
, 'spots_220717_wb3_sa1_B_9_5z18R_merfish5.csv': ['Zhuang-ABCA-3.009']
, 'spots_220609_wb3_sa1_1_5z18R_merfish5.csv': ['Zhuang-ABCA-3.001']
, 'spots_220627_wb3_sa1_6_5z18R_merfish6.csv': ['Zhuang-ABCA-3.006']
, 'spots_220613_wb3_sa1_2_5z18R_merfish5.csv': ['Zhuang-ABCA-3.002']
, 'spots_220506_sa_10_merfish4_adaptor_5z18r.csv': ['Zhuang-ABCA-3.010']
, 'spots_220606_wb3_sa1_3_5z18R_merfish6.csv': ['Zhuang-ABCA-3.003']
, 'spots_220710_wb3_sa1_B_7_5z18R_merfish5.csv': ['Zhuang-ABCA-3.007']
, 'spots_220424_sa_16_17_merfish4_adaptor_5z18r.csv': ['Zhuang-ABCA-3.017', 'Zhuang-ABCA-3.016']
, 'spots_220616_wb3_sa1_B_11_22_5z18R_merfish6.csv': ['Zhuang-ABCA-3.022', 'Zhuang-ABCA-3.011']
, 'spots_220421_sa_23_24_25_merfish4_adaptor_5z18r.csv': ['Zhuang-ABCA-3.023', 'Zhuang-ABCA-3.024']
, 'spots_220612_sa_15_18B_merfish4_adaptor.csv': ['Zhuang-ABCA-3.015']
, 'spots_220608_sa_4_merfish4_adaptor.csv': ['Zhuang-ABCA-3.004']
, 'spots_220620_sa_14_19B_merfish4_adaptor.csv': ['Zhuang-ABCA-3.019']
}

In [ ]:
for k in section_dict.keys():
    # for k, v in section_dict.items():
    #     if section in v:
    #         break
    print(k)
    path = '/mnt/Data18Td/Data/haichao/merfish_raw_data_zxw3/'
    decode = pd.read_csv(path + f'decode/{k}')
    decode_list = []
    for z in range(5):
        decode_list.append(decode[decode['global_z'] == z])
    decode = pd.concat(decode_list)

    label_list = []
    for z in range(5):
        label_tmp = pd.read_csv(f'./RNA_assign_result/z{z}_{k}')
        label_list.append(label_tmp)
    label = pd.concat(label_list)

    decode['cell_id'] = label['cell_id'].values

    in_cell = decode[(decode['cell_id'] != '-1') & (decode['cell_id'].isin(raw_adata.obs_names))]
    cell_mean_xy = in_cell.groupby('cell_id').agg({
        'global_x': 'mean',
        'global_y': 'mean'
    })
    plt.scatter(cell_mean_xy['global_x'], cell_mean_xy['global_y'], s=1)
    plt.show()
    out_cell_rna = decode[decode['cell_id'] == '-1']
    out_cell_rna = out_cell_rna[~out_cell_rna['target_gene'].str.startswith('blank-')]
    print('raw out_cell_RNA:')
    plt.scatter(out_cell_rna['global_x'], out_cell_rna['global_y'], s=0.1)
    plt.show()


    # start_time = time.time()
    coordinate = cell_mean_xy.values
    hull = ConvexHull(coordinate)
    polygon = Polygon(coordinate[hull.vertices])
    prepared_polygon = prep(polygon)
    min_x = coordinate[:, 0].min()
    max_x = coordinate[:, 0].max()
    min_y = coordinate[:, 1].min()
    max_y = coordinate[:, 1].max()
    out_cell_rna = out_cell_rna[(out_cell_rna['global_x'] >= min_x) & (out_cell_rna['global_x'] <= max_x) & (out_cell_rna['global_y'] >= min_y) & (out_cell_rna['global_y'] <= max_y)]
    points_to_check = np.array(out_cell_rna[['global_x', 'global_y']])
    in_hull = np.array([prepared_polygon.contains(Point(point)) for point in points_to_check])
    out_cell_rna['in_hull'] = in_hull.astype(int)
    out_cell_rna = out_cell_rna[out_cell_rna['in_hull'] == 1]
    print('after filter out_cell_RNA:')
    plt.scatter(out_cell_rna['global_x'], out_cell_rna['global_y'], s=0.1)
    plt.show()
    # end_time = time.time()
    # print(f"运行时间: {end_time - start_time} 秒")

    coordinate = cell_mean_xy.values
    out_cell_coordinate = out_cell_rna[['global_x', 'global_y']].values
    kdtree = cKDTree(coordinate)
    idx = []
    for coord in tqdm(out_cell_coordinate):
        # 查询最近邻索引
        _, nearest_index = kdtree.query(coord)
        idx.append(nearest_index)
    out_cell_rna['cell_name'] = list(cell_mean_xy.index[idx])

    # 去掉距离细胞太远的细胞外的RNA
    # start_time = time.time()
    out_cell_rna.rename(columns={'global_x': 'global_x_out', 'global_y': 'global_y_out'}, inplace=True)
    out_cell_rna = pd.merge(out_cell_rna, cell_mean_xy, left_on='cell_name', right_index=True, how='left')
    out_cell_rna['distance'] = np.sqrt((out_cell_rna['global_x_out'] - out_cell_rna['global_x']) ** 2 + (out_cell_rna['global_y_out'] - out_cell_rna['global_y']) ** 2)
    threshold = out_cell_rna['distance'].quantile(0.30)
    print(f'distance threshold:{threshold}')
    out_cell_rna = out_cell_rna[out_cell_rna['distance'] <= threshold]
    # print('after filter out_cell_RNA:')
    # plt.scatter(out_cell_rna['global_x'], out_cell_rna['global_y'], s=0.1)
    # plt.show()
    # end_time = time.time()
    # print(f"运行时间: {end_time - start_time} 秒")

    bininfo_exclude = out_cell_rna.groupby(['target_gene', 'cell_name']).size().reset_index(name='umi_count')
    out_cell_mean_xy = out_cell_rna.groupby(['cell_name'])[['x', 'y']].mean().reset_index()  # [3587 rows x 2 columns]

    obs = bininfo_exclude['cell_name'].unique()  # 35638677 -> 35369                   !!!!!!!!!!!!!!!!
    var = bininfo_exclude['target_gene'].unique()
    shape = (len(obs), len(var))
    obs_cat = CategoricalDtype(categories=sorted(obs), ordered=True)  # 35369
    var_cat = CategoricalDtype(categories=sorted(var), ordered=True)  # 22574
    obs_index = bininfo_exclude["cell_name"].astype(obs_cat).cat.codes  # 35638677          !!!!!!!!!!!!!!!!
    var_index = bininfo_exclude["target_gene"].astype(var_cat).cat.codes  # 35638677
    coo = sparse.coo_matrix((bininfo_exclude["umi_count"], (obs_index, var_index)), shape=shape)
    expcounts = coo.tocsr()

    # to anndata
    adata = ad.AnnData(expcounts)
    adata.obs_names = obs_cat.categories
    adata.var_names = var_cat.categories
    adata.obsm["spatial"] = out_cell_mean_xy[['x', 'y']].values
    adata.obs['totalRNA'] = adata.X.sum(1)

    adata.obs[['brain_section_label', 'x', 'y', 'z']] = raw_adata.obs.loc[adata.obs.index][['brain_section_label', 'x', 'y', 'z']]
    plt.show()
    plt.scatter(adata.obs['x'], adata.obs['y'], s=1, c=adata.obs['totalRNA'])
    if len(section_dict[k]) == 2:
        tmp1 = adata[adata.obs['brain_section_label'] == section_dict[k][0]]
        tmp2 = adata[adata.obs['brain_section_label'] == section_dict[k][1]]
        plt.scatter(tmp1.obs['x'], tmp1.obs['y'], s=1, c=tmp1.obs['totalRNA'])
        plt.show()
        plt.scatter(tmp2.obs['x'], tmp2.obs['y'], s=1, c=tmp2.obs['totalRNA'])
        plt.show()
        tmp1.write_h5ad(f'/mnt/Data18Td/Data/haichao/merfish_raw_data_zxw3/out_cell_adata/adata_out_cell_distance_q0.3/{section_dict[k][0]}.h5ad')
        tmp2.write_h5ad(f'/mnt/Data18Td/Data/haichao/merfish_raw_data_zxw3/out_cell_adata/adata_out_cell_distance_q0.3/{section_dict[k][-1]}.h5ad')
    else:
        adata.write_h5ad(f'/mnt/Data18Td/Data/haichao/merfish_raw_data_zxw3/out_cell_adata/adata_out_cell_distance_q0.3/{section_dict[k][0]}.h5ad')